<a href="https://colab.research.google.com/github/otoperalias/Coyuntura/blob/main/clases/Tema3_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 3.1. Manipulación de series

Leer las diapositivas disponibles en el [Aula Virtual](https://campusvirtual.upo.es/) antes de ejecutar este notebook.

In [ ]:
# Importamos funciones y establecemos configuración general
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('Solarize_Light2')

In [ ]:
# Importamos datos
tabla = pd.read_csv("https://raw.githubusercontent.com/otoperalias/Coyuntura/main/clases/datos/S%26P500.csv",sep=",", index_col=0, parse_dates=True)
sp500=tabla["Adj_Close"].copy()
sp500

#### ELIMINACIÓN DE LA TENDENCIA

In [ ]:
# Gráfico serie original
fig,ax=plt.subplots(1,1,figsize=(12,6))
ax.plot(sp500) # sp500.plot(ax=ax)
ax.set_title("S&P 500 Index - Adjusted Close Price - 1990-2019")
plt.show()

In [ ]:
# 1. Restando la línea de mejor ajuste calculada a través de una regresión
from scipy import signal
detrended = signal.detrend(sp500)
fig,ax=plt.subplots(1,1,figsize=(12,6))
ax.plot(detrended)
ax.set_title('Evolución de S&P eliminando la tendencia lineal', fontsize=16)
plt.show()

In [ ]:
# 2. Restar el componente de tendencia obtenido de la descomposición de series (lo veremos con más detalle en la sección 2 del tema)
from statsmodels.tsa.seasonal import seasonal_decompose
result_mul = seasonal_decompose(sp500, model='additive')
detrended = sp500 - result_mul.trend
fig,ax=plt.subplots(1,1,figsize=(12,6))
ax.plot(detrended)
ax.set_title('Evolución de S&P eliminado el componente tendencia de la desestacionalización', fontsize=16)
plt.show()

In [ ]:
# 3. Restando la media móvil
rol_mean=sp500.rolling(window=12, center=True).mean()
detrended = sp500 - rol_mean
fig,ax=plt.subplots(1,1,figsize=(12,6))
ax.plot(detrended)
ax.set_title('Evolución de S&P eliminado la media móvil', fontsize=16)
plt.show()

In [ ]:
# 4. Restando filtro HP
import statsmodels.api as sm
cycle, trend = sm.tsa.filters.hpfilter(sp500, 129600)
fig,ax=plt.subplots(1,1,figsize=(12,6))
ax.plot(cycle) # cycle es equivalente a  (sp500 - trend)
ax.set_title('Evolución de S&P eliminado la tendencia HP', fontsize=16)
plt.show()

#### ELIMINACIÓN COMPONENTE ESTACIONAL


In [ ]:
# Métodos de desestacionalización: lo veremos en la parte 2 del tema.

# Métodos sencillos
    # a Media móvil
desest=sp500.rolling(window=12).mean()
desest.plot()
    # b Diferencia interanual
desest=sp500.diff(12)
desest.plot()

#### TRATAMIENTO DE VALORES NULOS

In [ ]:
# Importamos librerías adicionales y datos nuevos con valores establecidos como nulos adrede.
from sklearn.metrics import mean_squared_error
tabla2= pd.read_csv("https://raw.githubusercontent.com/otoperalias/Coyuntura/main/clases/datos/S%26P500_con_nulos.csv",sep=",", index_col=0, parse_dates=True)
sp500_n=tabla2["Adj_Close"].copy()

# Seleccionamos las 100 últimas observaciones
sp500=sp500[-100:]
sp500_n=sp500_n[-100:]

In [ ]:
# Serie original resaltando valores nulos
plt.figure(figsize=(6,3), dpi=110)
plt.plot(sp500, color='red')
plt.plot(sp500_n, color='green')
plt.legend(["Missing Data", "Available Data"])

In [ ]:
# 1 Relleno hacia delante.
imputation = sp500_n.ffill()
error = mean_squared_error(sp500, imputation) # Para comparar luego qué método genera menos errores
print(error)
plt.plot(sp500, label='Original', color='green')
plt.plot(imputation, label='Imputation', color='red')
plt.legend()

In [ ]:
# 2 Relleno hacia atrás
imputation = sp500_n.bfill()
error = mean_squared_error(sp500, imputation)
print(error)
plt.plot(sp500, label='Original', color='green')
plt.plot(imputation, label='Imputation', color='red')
plt.legend()

In [ ]:
# 3. Interpolación lineal
imputation = sp500_n.interpolate()
error = mean_squared_error(sp500, imputation)
print(error)
plt.plot(sp500, label='Original', color='green')
plt.plot(imputation, label='Imputation', color='red')
plt.legend()

In [ ]:
# 4. Interpolación cúbica
imputation = sp500_n.interpolate(method="cubic")
error = mean_squared_error(sp500, imputation)
print(error)
plt.plot(sp500, label='Original', color='green')
plt.plot(imputation, label='Imputation', color='red')
plt.legend()

#### SUAVIZADO DE SERIES TEMPORALES

In [ ]:
# Importamos librería adicional y los datos de nuevo
from statsmodels.nonparametric.smoothers_lowess import lowess
tabla = pd.read_csv("https://raw.githubusercontent.com/otoperalias/Coyuntura/main/clases/datos/S%26P500.csv",sep=",", index_col=0, parse_dates=True)

In [ ]:
# 1. Moving Average
sp500_ma = tabla.Adj_Close.rolling(7, center=True).mean()

In [ ]:
# 2. Loess Smoothing (5% and 15%)
sp500_loess_2 = pd.DataFrame(lowess(tabla.Adj_Close, np.arange(len(tabla.Adj_Close)), frac=0.02)[:, 1], index=tabla.index, columns=['Adj_Close'])
sp500_loess_5 = pd.DataFrame(lowess(tabla.Adj_Close, np.arange(len(tabla.Adj_Close)), frac=0.05)[:, 1], index=tabla.index, columns=['Adj_Close'])

In [ ]:
# 3. Filtro HP
cycle, trend = sm.tsa.filters.hpfilter(tabla.Adj_Close, 14400)

In [ ]:
# Plot
fig, ax = plt.subplots(5,1, figsize=(7, 9), sharex=True, dpi=120)
fig.suptitle('Suavizado de series temporales', y=0.95, fontsize=14)
ax[0].plot(tabla.Adj_Close)
ax[0].set_title("Original", size="10")
ax[1].plot(sp500_loess_2)
ax[1].set_title("Loess smoothed 2%", size="10")
ax[2].plot(sp500_loess_5)
ax[2].set_title("Loess smoothed 5%", size="10")
ax[3].plot(sp500_ma)
ax[3].set_title("Media móvil", size="10")
ax[4].plot(trend)
ax[4].set_title("HP", size="10")
plt.subplots_adjust(hspace=0.25)
plt.show()